Pre-processing

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
  """
  Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
  it for training  
  """
  # Load the raw CIFAR-10 data
  cifar10_dir = 'cifar-10-batches-py'
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

  # Subsample the data
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]

  ###########################################################################
  # TODO: Implement the data pre-processing              #
  ###########################################################################
  mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
  std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
  X_train = (X_train - mean_pixel) / std_pixel
  X_val = (X_val - mean_pixel) / std_pixel
  X_test = (X_test - mean_pixel) / std_pixel
  ###########################################################################
  #             END OF YOUR CODE              #
  ###########################################################################
  
  return X_train, y_train, X_val, y_val, X_test, y_test

Model definition

In [ ]:
class LeNet(tf.keras.Model):
  def __init__(self, channel_1, channel_2, fc_1, fc_2, num_classes, reg1, reg2):
    super(LeNet, self).__init__()

    initializer = tf.initializers.VarianceScaling(scale=2.0)
    self.conv1 = tf.keras.layers.Conv2D(channel_1, [5,5], [1,1], padding='valid',
                                     kernel_initializer=initializer,
                                     kernel_regularizer=tf.keras.regularizers.l2(reg1))
    self.conv2 = tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='valid',
                                     kernel_initializer=initializer,
                                     kernel_regularizer=tf.keras.regularizers.l2(reg2))
    self.fc1 = tf.keras.layers.Dense(fc_1, kernel_initializer=initializer)
    self.fc2 = tf.keras.layers.Dense(fc_2, kernel_initializer=initializer)
    self.fc3 = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer)
    self.flatten = tf.keras.layers.Flatten()   
    self.softmax = tf.keras.layers.Softmax()
    self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='valid')
    
  def call(self, x, training=False):
    scores = None

    padding = tf.constant([[0,0],[2,2],[2,2],[0,0]])
    x = tf.pad(x, padding, 'CONSTANT')
    x = self.conv1(x)
    x = self.pool(x)
    padding = tf.constant([[0,0],[1,1],[1,1],[0,0]])
    x = tf.pad(x, padding, 'CONSTANT')
    x = self.conv2(x)
    x = self.pool(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    scores = self.softmax(x)
       
    return scores

def model_init_fn():
  
  channel_1, channel_2, fc_1, fc_2, num_classes = 6, 16, 120, 84, 10
  reg1, reg2 = 1e-2, 1e-2
  model = LeNet(channel_1, channel_2, fc_1, fc_2, num_classes, reg1, reg2)

  return model

Unzip trained-5.zip. Load the pretrained model and run testing. The testing accuracy is around 56%.



In [1]:
model = hub.KerasLayer("trained-5")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
for test_x, test_y in test_dset:
  prediction = model(test_x, training=False)
  test_accuracy.update_state(test_y, prediction)
print ('Test Accuracy: {}'.format(test_accuracy.result()*100))

NameError: name 'hub' is not defined